

# Create an interface with ZSL and relax it using EMT potentials

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials with minimal strain and then relax the resulting interface using the EMT potentials.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set slab parameters for the substrate and film in cell 2.1. (or use default).
1. Set interface parameters in cell 3.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create substrate and film slabs and select the terminations
1. Generate interfaces with strain matcher and plot strain vs number of atoms 
1. Select the interface with the desired strain and visualize it
1. Perform relaxation of the interface with set parameters
1. View the structure before and after relaxation

## Notes
1. We perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered.
1. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. 
1. ZSL strain matching is performed using Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).
1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [18]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 
# Maximum area for the superlattice search algorithm
MAX_AREA = 50

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [19]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [20]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
substrate = materials[0]
film = materials[1]

Data from 0-Ni has been read successfully.
Data from 1-Graphene has been read successfully.
Data from 2-WS2 has been read successfully.
Data from 3-BN has been read successfully.
Data from 4-Te2Mo has been read successfully.
Data from 5-HfO2 has been read successfully.
Data from C2(001)-Ni4(111) Interface, Strain (mean_abs_strain=0.001%) has been read successfully.
Data from C2(001)-Ni4(111) Interface has been read successfully.
Data from C2(001)-Ni4(111), Interface, Strain 0.001% has been read successfully.
Data from C2(001)-Ni4(111), Interface, Strain 0.105% has been read successfully.
Data from C2(001)-Ni4(111), Interface has been read successfully.
Data from Ni3 C2 has been read successfully.
Data from Si4 has been read successfully.
Data from Si8 has been read successfully.


### 1.4. Preview Substrate and Film

In [21]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

GridBox(children=(VBox(children=(Label(value='Ni - Material - rotation: 0x', layout=Layout(align_self='center'…

## 2. Configure slabs and select termination pair

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [22]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab

film_slab_configuration = SlabConfiguration(
    bulk=film,
    miller_indices=(0, 0, 1),
    thickness=1, # in atomic layers
    vacuum=0, # in atomic layers
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    miller_indices=(1,1,1),
    thickness=3, # in atomic layers
    vacuum=3, # in atomic layers
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

### 2.2. Get possible terminations for the slabs

In [23]:
film_slab_terminations = get_terminations(film_slab_configuration)
substrate_slab_terminations = get_terminations(substrate_slab_configuration)

### 2.3. Visualize slabs for all possible terminations

In [24]:
film_slabs = [create_slab(film_slab_configuration, termination) for termination in film_slab_terminations]
substrate_slabs = [create_slab(substrate_slab_configuration, termination) for termination in substrate_slab_terminations]

visualize([{"material":slab, "title": slab.metadata["termination"]} for slab in film_slabs ], repetitions=[3, 3, 1], rotation="-90x")
visualize([{"material":slab, "title": slab.metadata["termination"]} for slab in substrate_slabs ], repetitions=[3, 3, 1], rotation="-90x")  

GridBox(children=(VBox(children=(Label(value='C2 - C_P6/mmm_2 - rotation: -90x', layout=Layout(align_self='cen…

GridBox(children=(VBox(children=(Label(value='Ni3 - Ni_P6/mmm_1 - rotation: -90x', layout=Layout(align_self='c…

### 2.4. Print terminations for the interface

In [25]:
from itertools import product

termination_pairs = list(product(film_slab_terminations, substrate_slab_terminations))    
print("Termination Pairs (Film, Substrate)")
for idx, termination_pair in enumerate(termination_pairs):
    print(f"    {idx}: {termination_pair}")

Termination Pairs (Film, Substrate)
    0: (C_P6/mmm_2, Ni_P6/mmm_1)


### 2.5. Select termination pair for the interface

In [26]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

# Set the termination pair indices
TERMINATION_PAIR_INDEX = 0

termination_pair = termination_pairs[TERMINATION_PAIR_INDEX]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair = await ui_prompt_select_array_element_by_index_pyodide(termination_pairs, element_name="film/substrate termination pair")
    else:
        termination_pair = ui_prompt_select_array_element_by_index(termination_pairs, element_name="film/substrate termination pair")

## 3. Create interfaces

### 3.1. Initialize the Interface Configuration

In [27]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration

film_termination, substrate_termination = termination_pair
interface_configuration = InterfaceConfiguration(
    film_configuration=film_slab_configuration,
    substrate_configuration=substrate_slab_configuration,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance=3.0 # in Angstrom
)

### 3.2. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [28]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
zsl_strain_matching_parameters = ZSLStrainMatchingParameters(
    max_area=MAX_AREA
)

### 3.3. Generate interfaces with strain matcher
Interfaces are sorted by size and strain.

In [29]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingInterfaceBuilder, ZSLStrainMatchingInterfaceBuilderParameters

matched_interfaces_builder = ZSLStrainMatchingInterfaceBuilder(build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(strain_matching_parameters=zsl_strain_matching_parameters))

interfaces_sorted_by_size_and_strain= matched_interfaces_builder.get_materials(configuration=interface_configuration)

Interface will be built with terminations:  (C_P6/mmm_2, Ni_R-3m_1)


### 3.4. Plot interfaces by size and strain


In [30]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interfaces_sorted_by_size_and_strain, PLOT_SETTINGS)

### 3.5. Select the interface to relax

Select the index for the interface with the lowest strain and the smallest number of atoms.

In [31]:
# select the first interface with the lowest strain and the smallest number of atoms
interface_index = 0
interface = interfaces_sorted_by_size_and_strain[interface_index]

## 4. Preview the selected material

In [32]:
visualize(interface, repetitions=[3, 3, 1])
visualize(interface, repetitions=[3, 3, 1], rotation="-90x")

GridBox(children=(VBox(children=(Label(value='Ni3C2 - Material - rotation: 0x,0y,0z', layout=Layout(align_self…

GridBox(children=(VBox(children=(Label(value='Ni3C2 - Material - rotation: -90x', layout=Layout(align_self='ce…

## 5. Perform Relaxation
### 5.1. Set Relaxation Parameters

In [44]:
RELAXATION_PARAMETERS = {
    "FMAX": 0.018,
}

from mat3ra.made.tools.modify import RelaxationSettings, CalculatorEnum, OptimizerEnum
relaxation_settings = RelaxationSettings()

relaxation_settings.optimizer = OptimizerEnum.BFGS
relaxation_settings.calculator = CalculatorEnum.EMT
relaxation_settings.fmax = 0.05

### 5.2. Apply relaxation to the selected interface

In [45]:
import plotly.graph_objs as go
from IPython.display import display
from plotly.subplots import make_subplots
from mat3ra.made.tools.convert import to_ase, from_ase
from ase.optimize import BFGS
from ase.calculators.emt import EMT


# Set up the calculator 
calculator = EMT()

# Set up the interface for relaxation
ase_interface = to_ase(interface)
ase_interface.set_calculator(calculator)

dyn = BFGS(ase_interface)


# Initialize empty lists to store steps and energies
steps = []
energies = []

# Create a plotly figure widget
fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter"}]])
scatter = go.Scatter(x=[], y=[], mode='lines+markers', name='Energy')
fig.add_trace(scatter)
fig.update_layout(title_text='Real-time Optimization Progress', xaxis_title='Step', yaxis_title='Energy (eV)')

# Display figure widget
f = go.FigureWidget(fig)
display(f)


# Define a callback function to update the plot at each step
def plotly_callback():
    step = dyn.nsteps
    energy = ase_interface.get_total_energy()

    # Add the new step and energy to the lists
    steps.append(step)
    energies.append(energy)

    print(f"Step: {step}, Energy: {energy:.4f} eV")

    # Update the figure with the new data
    with f.batch_update():
        f.data[0].x = steps
        f.data[0].y = energies


# Run the relaxation
dyn.attach(plotly_callback, interval=1)
dyn.run(fmax=RELAXATION_PARAMETERS["FMAX"])

# Extract results
ase_original_interface = to_ase(interface)
ase_original_interface.set_calculator(calculator)
ase_final_interface = ase_interface
final_interface = from_ase(ase_final_interface)

original_energy = ase_original_interface.get_total_energy()
relaxed_energy = ase_interface.get_total_energy()

# Print out the final relaxed structure and energy
print(f"The final energy is {float(relaxed_energy):.3f} eV.")

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Energy',
              'type': 'scatter',
              'uid': 'c495c2f2-3f0b-4b7d-94a2-f63cc6b25c58',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Real-time Optimization Progress'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Step'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Energy (eV)'}}}
})

      Step     Time          Energy         fmax
BFGS:    0 17:03:00        1.445681        0.1415
Step: 0, Energy: 1.4457 eV
BFGS:    1 17:03:00        1.445033        0.1302
Step: 1, Energy: 1.4450 eV
BFGS:    2 17:03:00        1.432713        0.4527
Step: 2, Energy: 1.4327 eV
BFGS:    3 17:03:00        1.423774        0.6641
Step: 3, Energy: 1.4238 eV
BFGS:    4 17:03:00        1.359592        1.2474
Step: 4, Energy: 1.3596 eV
BFGS:    5 17:03:00        1.226410        1.4583
Step: 5, Energy: 1.2264 eV
BFGS:    6 17:03:00        1.502803        3.4047
Step: 6, Energy: 1.5028 eV
BFGS:    7 17:03:00        1.195866        1.3429
Step: 7, Energy: 1.1959 eV
BFGS:    8 17:03:00        1.173834        1.2495
Step: 8, Energy: 1.1738 eV
BFGS:    9 17:03:00        1.051097        1.4582
Step: 9, Energy: 1.0511 eV
BFGS:   10 17:03:00        0.997766        0.8415
Step: 10, Energy: 0.9978 eV
BFGS:   11 17:03:00        0.949498        0.7572
Step: 11, Energy: 0.9495 eV
BFGS:   12 17:03:00      

### 7.2. View structure before and after relaxation

In [46]:
visualize(interface, title="original", rotation="-90x")
visualize(final_interface, title="relaxed", rotation="-90x")



GridBox(children=(VBox(children=(Label(value='Ni3C2 - original - rotation: -90x', layout=Layout(align_self='ce…

Invalid material entry: {'name': 'Ni3 C2', 'basis': {'elements': [{'id': 0, 'value': 'Ni'}, {'id': 1, 'value': 'Ni'}, {'id': 2, 'value': 'Ni'}, {'id': 3, 'value': 'C'}, {'id': 4, 'value': 'C'}], 'coordinates': [{'id': 0, 'value': [0.666666666, 0.666666666, 0.370003259]}, {'id': 1, 'value': [1.0, 0.0, 0.444437319]}, {'id': 2, 'value': [0.333333335, 0.333333335, 0.518923746]}, {'id': 3, 'value': [0.333333332, 0.333333332, 0.590885865]}, {'id': 4, 'value': [0.666666667, 0.666666667, 0.575749812]}], 'units': 'crystal', 'cell': [[2.478974, 0.0, 0.0], [1.239487, 2.146854459, 0.0], [0.0, 0.0, 27.048147591]], 'constraints': []}, 'lattice': {'a': 2.478974, 'b': 2.478974, 'c': 27.048147591, 'alpha': 90.0, 'beta': 90.0, 'gamma': 60.0, 'units': {'length': 'angstrom', 'angle': 'degree'}, 'type': 'TRI', 'vectors': {'a': [2.478974, 0.0, 0.0], 'b': [1.239487, 2.146854459, 0.0], 'c': [0.0, 0.0, 27.048147591], 'alat': 1, 'units': 'angstrom'}}, 'isNonPeriodic': False, '_id': '', 'metadata': {'boundaryCon

GridBox(layout=Layout(grid_gap='10px', grid_template_columns='repeat(3, minmax(100px, 33.333333333333336%))', …

### 7.3. Calculate energy energy using ASE EMT

In [ ]:
def filter_atoms_by_tag(atoms, material_index):
    """Filter atoms by their tag, corresponding to the material index."""
    return atoms[atoms.get_tags() == material_index]


def calculate_energy(atoms, calculator):
    """Set calculator for atoms and return their total energy."""
    atoms.set_calculator(calculator)
    return atoms.get_total_energy()


def calculate_delta_energy(total_energy, *component_energies):
    """Calculate the delta energy by subtracting component energies from the total energy."""
    return total_energy - sum(component_energies)


# Filter atoms for original and relaxed interfaces
substrate_original_interface = filter_atoms_by_tag(ase_original_interface, 0)
layer_original_interface = filter_atoms_by_tag(ase_original_interface, 1)
substrate_relaxed_interface = filter_atoms_by_tag(ase_final_interface, 0)
layer_relaxed_interface = filter_atoms_by_tag(ase_final_interface, 1)

# Calculate energies
original_substrate_energy = calculate_energy(substrate_original_interface, calculator)
original_layer_energy = calculate_energy(layer_original_interface, calculator)
relaxed_substrate_energy = calculate_energy(substrate_relaxed_interface, calculator)
relaxed_layer_energy = calculate_energy(layer_relaxed_interface, calculator)

# Calculate delta energies
delta_original = calculate_delta_energy(original_energy, original_substrate_energy, original_layer_energy)
delta_relaxed = calculate_delta_energy(relaxed_energy, relaxed_substrate_energy, relaxed_layer_energy)

# Calculate area and effective delta per area
area = ase_original_interface.get_volume() / ase_original_interface.cell[2, 2]
number_of_interface_atoms = ase_final_interface.get_global_number_of_atoms()
number_of_substrate_atoms = substrate_relaxed_interface.get_global_number_of_atoms()
number_of_layer_atoms = layer_relaxed_interface.get_global_number_of_atoms()
effective_delta_relaxed = (relaxed_energy/number_of_interface_atoms - (relaxed_substrate_energy/number_of_substrate_atoms + relaxed_layer_energy/number_of_layer_atoms)) / (2 * area)

# Print out the metrics
print(f"Original Substrate energy: {original_substrate_energy:.4f} eV")
print(f"Relaxed Substrate energy: {relaxed_substrate_energy:.4f} eV")
print(f"Original Layer energy: {original_layer_energy:.4f} eV")
print(f"Relaxed Layer energy: {relaxed_layer_energy:.4f} eV")
print("\nDelta between interface energy and sum of component energies")
print(f"Original Delta: {delta_original:.4f} eV")
print(f"Relaxed Delta: {delta_relaxed:.4f} eV")
print(f"Original Delta per area: {delta_original / area:.4f} eV/Ang^2")
print(f"Relaxed Delta per area: {delta_relaxed / area:.4f} eV/Ang^2")
print(f"Relaxed interface energy: {relaxed_energy:.4f} eV")
print(f"Effective relaxed Delta per area: {effective_delta_relaxed:.4f} eV/Ang^2 ({effective_delta_relaxed / 0.16:.4f} J/m^2)\n")

# Print out the POSCARs
print("Relaxed interface:\n", ase_to_poscar(ase_final_interface))
print("Relaxed substrate:\n", ase_to_poscar(substrate_relaxed_interface))
print("Relaxed layer:\n", ase_to_poscar(layer_relaxed_interface))